In [47]:
from os import listdir
from os.path import isfile, isdir, join
from xml.etree import ElementTree as etree
import pandas as pd
import gzip
import time
import csv

# Decompress _.tar.gz_ Files

In [48]:
# Decompress Files
def decompress(myPath):
    fileNames = [f for f in listdir(myPath) if isfile(join(myPath, f)) and f[-7:] == ".tar.gz"]
    for file in fileNames:
        tar = tarfile.open(join(myPath,file), "r:gz")
        tar.extractall(join(myPath,file[:-7]))
        tar.close()
        
if __name__ == "__main__":
    # Path containing the medline_xml.part[x].tar.gz files
    pubMedAbstracts = "XML-Collection"

    # Decompress files
    decompress(pubMedAbstracts)

# Extract Information from Pubmed XML Files

In [50]:
# Extract Pubmed Ids from the Gold-Standard CSV file
def extractDocIDs(filePath):
    """ Extracts all ids from the gold standard """
    f = pd.read_csv(filePath)
    return f['trec_doc_id'].tolist()

# Get the name of the folders containing xml.gz files
def getFolderNames(myPath):
    dirNames = [d for d in listdir(myPath) if isdir(join(myPath, d))]
    return dirNames

# Get the name of the xml.gz files
def getGzFileNames(myPath):
    fileNames = [f for f in listdir(myPath) if isfile(join(myPath, f)) and f[-7:] == ".xml.gz"]
    return fileNames
        
# Extract relevant information from the papers inside the XML files that match the gold-standard
def extractFeatures(folderPath, docIDsPath, outputPath):
    st = time.time()
    
    # Get Pubmed Ids from the Gold-Standard
    ids = extractDocIDs(docIDsPath)
    print("Nr of PMIDs in the Gold-Standard:", len(ids))
    # Recover the names of each folder containing xml.gz files
    folderNames = getFolderNames(folderPath)
    
    nrExtractedXMLs = 0
    
    # Create CSV for the output
    with open(outputPath, 'w') as extractFile:
        wr = csv.writer(extractFile, quoting=csv.QUOTE_ALL, delimiter="\t")
        wr.writerow(["trec_doc_id","title","abstract","major_mesh","minor_mesh"])
    
    # Iterate through the folders with the xml.gz files
    for folderName in folderNames:
        print("Looking into files from folder: ", folderName)
        gzFiles = getGzFileNames(join(folderPath,folderName))
        for gzFileName in gzFiles:
            print("Analyzing information from: ", gzFileName)
            gzFilePath = join(join(folderPath,folderName), gzFileName)
            pubMedArticleSet = etree.parse(gzip.open(gzFilePath, 'rt')).getroot()
            for mc in pubMedArticleSet.iterfind('PubmedArticle/MedlineCitation'):
                pmid = mc.find("PMID").text
                # Verify if the PMID is in the list of IDs from the Gold-Standard
                majorMeshTerms = []
                minorMeshTerms = []
                if pmid in ids:
                    print("Extracting info from the PMID: ", pmid)
                    if mc.find("Article/ArticleTitle") is not None:
                        title = mc.find("Article/ArticleTitle").text
                    if mc.find("Article/Abstract/AbstractText") is not None:
                        abstract = mc.find("Article/Abstract/AbstractText").text
                    # Extracting major and minor mesh descriptors
                    # Qualifiers - not taking into account major and minor attributes
                    for meshTerm in mc.findall("MeshHeadingList/MeshHeading"):
                        qualifiers = []
                        for qualifier in meshTerm.findall("QualifierName"):
                            qualifiers.append(meshTerm.find("DescriptorName").text + "/" + qualifier.text)
                        if not qualifiers:
                            fullMesh = meshTerm.find("DescriptorName").text
                            if meshTerm.find("DescriptorName").get("MajorTopicYN") == "Y":
                                majorMeshTerms.append(fullMesh)
                            else:
                                minorMeshTerms.append(fullMesh)
                        else:
                            if meshTerm.find("DescriptorName").get("MajorTopicYN") == "Y":
                                majorMeshTerms.extend(qualifiers)
                            else:
                                minorMeshTerms.extend(qualifiers)
                    majorMeshList = ";".join(majorMeshTerms)
                    minorMeshList = ";".join(minorMeshTerms)
                    
                    # Write the result to CSV
                    with open(outputPath, 'a', encoding="utf8") as extractFile:
                        wr = csv.writer(extractFile, quoting=csv.QUOTE_ALL, delimiter="\t")
                        wr.writerow([pmid, title, abstract, majorMeshList, minorMeshList])
                
                    # Count the number of extracted papers
                    nrExtractedXMLs += 1
    
    print("Number of papers with information extracted: ", nrExtractedXMLs)
    end = time.time()
    print("Run time: ", end-st)

if __name__ == "__main__":
    # Path containing the medline_xml.partx folders - they need to be extracted first
    pubMedAbstracts = "XML-Collection"
    # Path containing the Annotated Gold-Standard File
    docIDPath = "goldDocsAnnotations.csv"
    # Output file
    outputPath = "relevantAbstractsFromXML.csv"
    
    # Extract relevant information from the XML files
    extractFeatures(pubMedAbstracts, docIDPath, outputPath)

Nr of PMIDs in the Gold-Standard: 22642
Looking into files from folder:  medline_xml.part1
	Analyzing information from:  medline17n0133.xml.gz
	Extracting info from the PMID:  3986769
	Extracting info from the PMID:  3998058
	Extracting info from the PMID:  4007423
	Analyzing information from:  medline17n0284.xml.gz
	Extracting info from the PMID:  8521195
	Extracting info from the PMID:  8522124
	Extracting info from the PMID:  8523820
	Extracting info from the PMID:  8524423
	Extracting info from the PMID:  8526197
	Extracting info from the PMID:  8528972
	Extracting info from the PMID:  8529465
	Extracting info from the PMID:  8529957
	Extracting info from the PMID:  8535808
	Extracting info from the PMID:  8538973
	Extracting info from the PMID:  8541113
	Extracting info from the PMID:  8541555
	Extracting info from the PMID:  8545128
	Extracting info from the PMID:  8546209
	Extracting info from the PMID:  8546466
	Extracting info from the PMID:  8548242
	Extracting info from the 

	Analyzing information from:  medline17n0195.xml.gz
	Analyzing information from:  medline17n0241.xml.gz
	Extracting info from the PMID:  7231886
	Extracting info from the PMID:  7235384
	Extracting info from the PMID:  7242845
	Extracting info from the PMID:  7244759
	Analyzing information from:  medline17n0187.xml.gz
	Analyzing information from:  medline17n0230.xml.gz
	Analyzing information from:  medline17n0087.xml.gz
	Extracting info from the PMID:  2611950
	Extracting info from the PMID:  2611995
	Extracting info from the PMID:  2612432
	Extracting info from the PMID:  2632688
	Extracting info from the PMID:  2633494
	Analyzing information from:  medline17n0199.xml.gz
	Analyzing information from:  medline17n0099.xml.gz
	Extracting info from the PMID:  2969039
	Extracting info from the PMID:  2983887
	Extracting info from the PMID:  2986619
	Extracting info from the PMID:  2986827
	Analyzing information from:  medline17n0253.xml.gz
	Extracting info from the PMID:  7590738
	Extractin

	Extracting info from the PMID:  3689662
	Extracting info from the PMID:  3689982
	Extracting info from the PMID:  3691405
	Extracting info from the PMID:  3697942
	Analyzing information from:  medline17n0298.xml.gz
	Extracting info from the PMID:  8949425
	Extracting info from the PMID:  8950667
	Extracting info from the PMID:  8956123
	Extracting info from the PMID:  8956159
	Extracting info from the PMID:  8957089
	Extracting info from the PMID:  8957203
	Extracting info from the PMID:  8957600
	Extracting info from the PMID:  8958263
	Extracting info from the PMID:  8958806
	Extracting info from the PMID:  8959678
	Extracting info from the PMID:  8960139
	Extracting info from the PMID:  8964150
	Extracting info from the PMID:  8968085
	Extracting info from the PMID:  8968226
	Extracting info from the PMID:  8968868
	Extracting info from the PMID:  8971180
	Extracting info from the PMID:  8977556
	Analyzing information from:  medline17n0152.xml.gz
	Analyzing information from:  medli

	Extracting info from the PMID:  9038605
	Extracting info from the PMID:  9040836
	Analyzing information from:  medline17n0200.xml.gz
	Analyzing information from:  medline17n0271.xml.gz
	Extracting info from the PMID:  8131454
	Extracting info from the PMID:  8134123
	Extracting info from the PMID:  8141431
	Extracting info from the PMID:  8143346
	Extracting info from the PMID:  8143615
	Extracting info from the PMID:  8147063
	Extracting info from the PMID:  8152265
	Extracting info from the PMID:  8157255
	Extracting info from the PMID:  8158700
	Extracting info from the PMID:  8158781
	Analyzing information from:  medline17n0212.xml.gz
	Extracting info from the PMID:  6367450
	Extracting info from the PMID:  6369971
	Extracting info from the PMID:  6371995
	Extracting info from the PMID:  6385716
	Analyzing information from:  medline17n0072.xml.gz
	Extracting info from the PMID:  2153943
	Extracting info from the PMID:  2154327
	Extracting info from the PMID:  2154855
	Extracting i

	Extracting info from the PMID:  1741644
	Extracting info from the PMID:  1743203
	Extracting info from the PMID:  1744438
	Extracting info from the PMID:  1744441
	Extracting info from the PMID:  1753501
	Analyzing information from:  medline17n0158.xml.gz
	Extracting info from the PMID:  4752347
	Analyzing information from:  medline17n0025.xml.gz
	Extracting info from the PMID:  758856
	Analyzing information from:  medline17n0292.xml.gz
	Extracting info from the PMID:  8763258
	Extracting info from the PMID:  8763514
	Extracting info from the PMID:  8768685
	Extracting info from the PMID:  8769719
	Extracting info from the PMID:  8775092
	Extracting info from the PMID:  8778232
	Extracting info from the PMID:  8778798
	Extracting info from the PMID:  8780392
	Extracting info from the PMID:  8780405
	Extracting info from the PMID:  8780719
	Extracting info from the PMID:  8781657
	Extracting info from the PMID:  8784474
	Extracting info from the PMID:  8785673
	Extracting info from the

...

	Extracting info from the PMID:  27005720
	Extracting info from the PMID:  27006248
	Extracting info from the PMID:  27006301
	Extracting info from the PMID:  27007084
	Extracting info from the PMID:  27009410
	Extracting info from the PMID:  27010960
	Extracting info from the PMID:  27012422
	Extracting info from the PMID:  27014910
	Extracting info from the PMID:  27015401
	Extracting info from the PMID:  27015600
	Extracting info from the PMID:  27016421
	Extracting info from the PMID:  27017063
	Extracting info from the PMID:  27017943
	Extracting info from the PMID:  27020204
	Extracting info from the PMID:  27020493
	Extracting info from the PMID:  27020606
	Extracting info from the PMID:  27022491
	Extracting info from the PMID:  27022754
	Extracting info from the PMID:  27026619
	Extracting info from the PMID:  27026675
	Extracting info from the PMID:  27027150
	Extracting info from the PMID:  27027665
	Extracting info from the PMID:  27027731
	Extracting info from the PMID:  2

	Extracting info from the PMID:  27571409
	Extracting info from the PMID:  27573754
	Extracting info from the PMID:  27573756
	Extracting info from the PMID:  27573812
	Extracting info from the PMID:  27573925
	Extracting info from the PMID:  27574435
	Extracting info from the PMID:  27575024
	Extracting info from the PMID:  27576846
	Extracting info from the PMID:  27576872
	Extracting info from the PMID:  27577079
	Extracting info from the PMID:  27578530
	Extracting info from the PMID:  27578983
	Extracting info from the PMID:  27579296
	Extracting info from the PMID:  27579736
	Extracting info from the PMID:  27580131
	Extracting info from the PMID:  27580550
	Extracting info from the PMID:  27581326
	Extracting info from the PMID:  27581327
	Extracting info from the PMID:  27581576
	Extracting info from the PMID:  27581627
	Extracting info from the PMID:  27586202
	Extracting info from the PMID:  27586264
	Analyzing information from:  medline17n0889.xml.gz
	Extracting info from th

	Extracting info from the PMID:  27453460
	Extracting info from the PMID:  27453739
	Extracting info from the PMID:  27454027
	Extracting info from the PMID:  27454293
	Extracting info from the PMID:  27454298
	Extracting info from the PMID:  27456296
	Analyzing information from:  medline17n0864.xml.gz
	Extracting info from the PMID:  27036725
	Extracting info from the PMID:  27037254
	Extracting info from the PMID:  27039821
	Extracting info from the PMID:  27039845
	Extracting info from the PMID:  27040854
	Extracting info from the PMID:  27040858
	Extracting info from the PMID:  27042173
	Extracting info from the PMID:  27044261
	Extracting info from the PMID:  27044426
	Extracting info from the PMID:  27044840
	Extracting info from the PMID:  27046225
	Extracting info from the PMID:  27047697
	Extracting info from the PMID:  27048113
	Extracting info from the PMID:  27048128
	Extracting info from the PMID:  27048246
	Extracting info from the PMID:  27048593
	Extracting info from th

	Extracting info from the PMID:  27261907
	Extracting info from the PMID:  27262212
	Extracting info from the PMID:  27262438
	Extracting info from the PMID:  27262570
	Extracting info from the PMID:  27262676
	Extracting info from the PMID:  27263309
	Analyzing information from:  medline17n0862.xml.gz
	Extracting info from the PMID:  26975494
	Extracting info from the PMID:  26976973
	Extracting info from the PMID:  26977001
	Extracting info from the PMID:  26977002
	Extracting info from the PMID:  26977004
	Extracting info from the PMID:  26977054
	Extracting info from the PMID:  26977877
	Extracting info from the PMID:  26977879
	Extracting info from the PMID:  26979245
	Extracting info from the PMID:  26980021
	Extracting info from the PMID:  26980032
	Extracting info from the PMID:  26980034
	Extracting info from the PMID:  26980572
	Extracting info from the PMID:  26980732
	Extracting info from the PMID:  26982510
	Extracting info from the PMID:  26983463
	Extracting info from th

	Extracting info from the PMID:  27590631
	Extracting info from the PMID:  27591291
	Extracting info from the PMID:  27591497
	Extracting info from the PMID:  27591990
	Extracting info from the PMID:  27594953
	Extracting info from the PMID:  27595804
	Extracting info from the PMID:  27595930
	Extracting info from the PMID:  27597273
	Extracting info from the PMID:  27597521
	Extracting info from the PMID:  27597738
	Extracting info from the PMID:  27597976
	Extracting info from the PMID:  27600832
	Extracting info from the PMID:  27601168
	Extracting info from the PMID:  27601837
	Extracting info from the PMID:  27602120
	Extracting info from the PMID:  27602167
	Extracting info from the PMID:  27602494
	Extracting info from the PMID:  27603005
	Extracting info from the PMID:  27603340
	Extracting info from the PMID:  27606136
	Extracting info from the PMID:  27606884
	Extracting info from the PMID:  27608486
	Extracting info from the PMID:  27608883
	Extracting info from the PMID:  2

# Read Relevant Abstracts Output

In [56]:
abstracts = pd.read_csv("relevantAbstractsFromXML.csv", sep='\t', encoding="utf-8", dtype={'trec_doc_id':object})
abstracts.shape

(17547, 5)

# Read Gold-Standard Annotation

In [57]:
annotations = pd.read_csv("goldDocsAnnotations.csv", sep=',', encoding="utf-8")
annotations.shape

(22642, 12)

# Read Relevance Score

In [58]:
relevance = pd.read_csv("qrels-treceval-abstracts.2017.txt", sep=' ', encoding="utf-8", header=None, 
                        names=["trec_topic_number", "x", "trec_doc_id", "relevance_score"])
relevance.shape

(22642, 4)

# Merge Relevance Score and Annotations

In [59]:
annotationsRelevance = annotations.merge(relevance, left_on=['trec_topic_number','trec_doc_id'], right_on=['trec_topic_number','trec_doc_id'], how='left')
annotationsRelevance.drop(["x"], axis=1, inplace=True)
annotationsRelevance.shape

(22642, 13)

# Merge Abstracts with Relevance Score and Annotations

In [68]:
processedGoldStandard = annotationsRelevance.merge(abstracts, left_on=['trec_doc_id'], right_on=['trec_doc_id'], how='left')
processedGoldStandard.shape


(22643, 17)

# Save the Result into a new _.csv_

In [70]:
processedGoldStandard.to_csv(path_or_buf='processedGoldStandard.csv', sep='\t')